# Data cleaning 

In [27]:
import pandas as pd
import os

In [2]:
from use_cases.contributions import create_table_contributions

from use_cases.emotions import create_table_emotions
from use_cases.personal_needs import create_table_personal_needs
from use_cases.dialogues import create_table_dialogues
from use_cases.country_needs import create_table_country_needs
from use_cases.persons import create_table_persons
from use_cases.individuals import create_table_individuals
from use_cases.pairs import create_pair_token


pd.options.mode.chained_assignment = None 

%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Loading Data Surveys

In [3]:
chunksize   = 100 # Just for debugging
survey_path = './data/BBDD_Dialogos_2021_01_27.csv' #  chunksize=chunksize,
survey = pd.read_csv(survey_path, low_memory=False)
# for survey in survey: break
    
ind_online_survey_path = './data/Consulta_Individual_online_v2.xlsx'
ind_online_survey = pd.read_excel(ind_online_survey_path, 'Sheet1')
    
ind_survey_path = './data/Base_final_digitación_Consultas.xlsx'
ind_survey = pd.read_excel(ind_survey_path, 'CONSULTAS')

In [4]:
survey.sample(2)
ind_online_survey.sample(2)

,Submission Date,RUN,Edad,Comuna,1. ¿Cuál es el nivel de educación alcanzado por Usted?,2. ¿Cuál es el nivel de educación que alcanzó el jefe de su hogar? (sólo si se trata de alguien distinto a Usted),1 >> Emociones / Sentimientos / Sensaciones,1 >> Explique lo mencionado,2 >> Emociones / Sentimientos / Sensaciones,2 >> Explique lo mencionado,...,2 >> Rol del actor social (Describa),3 >> Necesidades del país identificadas,3 >> Rol del Estado (Describa),"3 >> Actor social (empresa, organizaciones sociales, medios de comunicación, comunidad, etc)",3 >> Rol del actor social (Describa),5. Pregunta: ¿Cuál es mi contribución personal para construir el Chile que queremos?,Comuna.1,"Comentarios finales: Si tiene alguna observación o sugerencia, en este espacio puede indicarla.",Región,Región.1
381,2020/01/17 12:00:36,1649cf3ea0809f3049991295aac1fda1,35,NaN,Universitaria completa.,Universitaria completa.,enojada,me enoja la situacion que vivimos como pais,insegura,me molesta las situaciones de vandalismo,...,NaN,educacion,el estado debiera hacer una reforma en la educ...,las universidades no deberian pedir psu,NaN,Contribuyo con mi trabajo y con los impuestos ...,Punta Arenas,NaN,Región de Magallanes y Antártica Chilena,NaN
1243,2019/12/18 16:35:20,a49062d4bbfb7ec322045c2e036e97fb,38,Viña del Mar,Universitaria completa.,"Post Grado (Master, Doctor o equivalente)",Furiosa,Con ganas de patear al gobierno por ser tan en...,Frustrada,Porque votamos y elegimos a un presidente supu...,...,"Los colegios privados tienen que asegurarse, t...",Salud,El Estado tiene que preocuparse de sus centros...,Clínicas y otras entidades de salud privadas,Deben mantener su excelencia y trabajar más es...,"Voto en todas las elecciones, confiando en la ...",NaN,"Me gustaría que el presidente de Chile, Sebast...",NaN,Valparaíso


## Processing Uses Cases
1. Emotions 
2. Contributions
3. Country needs 
4. Personal/familiar needs

### Persons Table (Dialogue)

In [5]:
%%time
persons_table = create_table_persons(survey)

CPU times: user 1min 31s, sys: 55.8 ms, total: 1min 31s
Wall time: 1min 31s


In [6]:
persons_table.sample(3)

,id,diag_id,age,sex,level,comuna_id,age_range
229,51148,enc_u_4584412065317688074,61,f,tecnico_ completo,8301,>=60
9253,59699,enc_u_3301344,48,h,,1,45-60
10954,10589,enc_u_4564226989027123245,37,h,universitaria_incompleta,13106,30-45


### Dialogues

In [7]:
%%time
dialogues_table = create_table_dialogues(survey)

CPU times: user 1.61 s, sys: 0 ns, total: 1.61 s
Wall time: 1.61 s


In [8]:
dialogues_table.sample(3)

,id,date,init_time,end_time,location,address,comuna_id,n_members,group_name
12988,enc_u_4517191811489237194,09-12-2019,11:00:00,13:00:00,centro diurno adulto mayor,camino publico,13501,10,usuarios de oficina movil de bienes nacionales
7858,enc_u_4559459052499790303,23-01-2020,18:00:00,19:30:00,salon consejo municipal,plaza del cabildo s/n,5606,8,usuarios de oficina movil de bienes nacionales
10881,enc_u_4558651993122756353,22-01-2020,16:27:00,18:03:00,sede,avda central 0217,13201,5,usuarios de oficina movil de bienes nacionales


### Individuals

In [9]:
%%time
individuals_table = create_table_individuals(ind_online_survey, ind_survey)

CPU times: user 7.26 s, sys: 48.5 ms, total: 7.31 s
Wall time: 7.23 s


In [10]:
individuals_table.sample(2)

,id,date,age,comuna_id,level,online,age_range
5368,3234194,28-01-2020,73,10108,Universitaria incompleta. Técnica completa,,>=60
9809,5144614,,,1,,,0-15


### Emotions

In [11]:
%%time
emotions_table = create_table_emotions(survey, ind_survey_path, ind_online_survey)

CPU times: user 2min 9s, sys: 9.06 s, total: 2min 18s
Wall time: 2min 18s


In [12]:
emotions_table.sample(2)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,is_online,source_id
13056,60143,ENC_U_4544929856116003794,,camila\n(mal),"[camila, mal]",camila\n(mal),- como estudiante me ah afectado por el alza d...,"[estudiante, ah, afectado, alza, transporte, a...",0,
7253,55583,ENC_U_4548220530117013700,,estres.,[estres],estres.,por la violencia desplegada los ultimas semana...,"[violencia, desplegada, ultimas, semanas, part...",0,


### Emotion Pair

In [13]:
%%time
emotion_pair = create_pair_token(emotions_table, 'exp_tokens', 'emotion_id')

CPU times: user 10.6 s, sys: 48 ms, total: 10.7 s
Wall time: 10.7 s


In [14]:
emotion_pair.sample(3)

,id,emotion_id,word_1,word_2
68927,68928,12280,dolor,pais
169719,169720,30761,habran,cambios
50612,50613,9076,reventar,igual


### Country Needs

In [15]:
%%time
country_needs = create_table_country_needs(survey, ind_survey, ind_survey_path, ind_online_survey_path)

CPU times: user 4min 21s, sys: 15.5 s, total: 4min 36s
Wall time: 4min 37s


In [16]:
country_needs.sample(3)

,id,diag_id,ind_id,name,name_tokens,macro,exp,exp_tokens,role,role_tokens,actor,priority,is_online
10191,49296,enc_u_4555129598558603784,,afp,[afp],,malo \nmejorar la edad de la jubilacion\nhacia...,"[malo, mejorar, edad, jubilacion, hacia, anos]",,,,0,0
20958,77225,enc_u_4556921180157433493,,aumentar pensiones 3 edad,"[aumentar, pensiones, edad]",,,,parlamentarios,[parlamentarios],estado,,0
3458,80680,enc_u_4608704691193105727,,proteccion a la infancia,"[proteccion, infancia]",,mejorar la educacion familiar\n-seguimiento a ...,"[mejorar, educacion, familiar, seguimiento, ca...",-impartir clases y actividades para padres y f...,"[impartir, clases, actividades, padres, familia]",comunidad,5,0


### Country Need Role Pair

In [17]:
%%time
country_need_role_pair = create_pair_token(country_needs, 'role_tokens', 'country_need_id')

CPU times: user 13.7 s, sys: 56.1 ms, total: 13.7 s
Wall time: 13.7 s


In [18]:
country_need_role_pair.sample(2)

,id,country_need_id,word_1,word_2
76064,76065,24447,queden,toda
320333,320334,116939,parte,plan


### Country Need Explanation Pair

In [19]:
%%time
country_need_exp_pair = create_pair_token(country_needs, 'exp_tokens', 'country_need_id')

CPU times: user 14.3 s, sys: 72 ms, total: 14.4 s
Wall time: 14.4 s


In [20]:
country_need_exp_pair.sample(2)

,id,country_need_id,word_1,word_2
319255,319256,70216,mas,viviendas
72588,72589,11039,ser,calidad


### Family/Personal Needs

In [21]:
%%time
personal_needs = create_table_personal_needs(survey, ind_survey, ind_online_survey)

CPU times: user 2min 23s, sys: 10.3 s, total: 2min 33s
Wall time: 2min 33s


In [22]:
personal_needs.sample(3)

,id,diag_id,ind_id,name,name_tokens,exp,exp_tokens,macro,priority,is_online
4286,16069,ENC_U_4555386981336010252,,salud,[salud],mejorar la calidad de la atencion publica que ...,"[mejorar, calidad, atencion, publica, igualita...",salud,0.0,0
7863,19174,ENC_U_4559501678807239126,,pensiones,[pensiones],-las pensiones nos alcanza solo para nuestros ...,"[pensiones, alcanza, solo, gastos, basicos]",pensiones,2.0,0
5854,71584,,3247056,recibir mi sueldo siempre,"[recibir, sueldo, siempre]",,,recibir mi sueldo siempre,2.0,0


### Family/Personal Need Pair

In [23]:
%%time
personal_need_pair = create_pair_token(personal_needs, 'exp_tokens', 'personal_need_id')

CPU times: user 12.2 s, sys: 56 ms, total: 12.2 s
Wall time: 12.2 s


In [24]:
personal_need_pair.sample(3)

,id,personal_need_id,word_1,word_2
412575,412576,74901,trabajo,integrantes
10824,10825,1726,obligaciones,ley
82815,82816,13111,debiera,ser


### Contributions

In [25]:
%%time
contributions = create_table_contributions(survey, ind_survey, ind_online_survey)

CPU times: user 57.8 s, sys: 4.33 s, total: 1min 2s
Wall time: 1min 2s


In [26]:
contributions.sample(3)

,id,diag_id,ind_id,text,tokens,macro,is_online
2833,29481,ENC_U_4541418198643324428,,ir a votar,"[ir, votar]",ir a votar,0
343,40315,ENC_U_4575155340779421978,,aportar de manera activa en la sociedad,"[aportar, manera, activa, sociedad]",aportar de manera activa en la sociedad,0
1527,14851,ENC_U_4553457074807180068,,dando ideas,"[dando, ideas]",dando ideas,0


## Saving `.csv`

In [28]:
out_dir = './out'
os.makedirs(out_dir, exist_ok=True)

In [30]:
contributions.to_csv(os.path.join(out_dir, 'contributions.csv'), index=False)
personal_need_pair.to_csv(os.path.join(out_dir, 'personal_need_pair.csv'), index=False)
personal_needs.to_csv(os.path.join(out_dir, 'personal_needs.csv'), index=False)
country_need_exp_pair.to_csv(os.path.join(out_dir, 'country_need_exp_pair.csv'), index=False)
country_need_role_pair.to_csv(os.path.join(out_dir, 'country_need_role_pair.csv'), index=False)
country_needs.to_csv(os.path.join(out_dir, 'country_needs.csv'), index=False)
emotion_pair.to_csv(os.path.join(out_dir, 'emotion_pair.csv'), index=False)
emotions_table.to_csv(os.path.join(out_dir, 'emotions_table.csv'), index=False)
individuals_table.to_csv(os.path.join(out_dir, 'individuals_table.csv'), index=False)
dialogues_table.to_csv(os.path.join(out_dir, 'dialogues_table.csv'), index=False)
persons_table.to_csv(os.path.join(out_dir, 'persons_table.csv'), index=False)